In [1]:
# # Cell 1: Install dependencies
# # !pip install tensorflow opencv-python matplotlib

# # Cell 2: Imports
# import tensorflow as tf
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# from pathlib import Path

# # Cell 3: Load input image
# def load_image(path, target_size=(256, 256)):
#     img = cv2.imread(str(path))
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img_resized = cv2.resize(img, target_size)
#     img_norm = img_resized / 255.0  # normalize
#     return img_resized, np.expand_dims(img_norm, axis=0)  # original + batch format

# image_path = Path("input_image.jpg")  # replace with your own
# original_img, input_tensor = load_image(image_path)

# # Cell 4: Show input
# plt.figure(figsize=(6, 6))
# plt.imshow(original_img)
# plt.title("Input Image")
# plt.axis("off")
# plt.show()

# # Cell 5: Load pretrained segmentation model
# # Replace with your actual model path
# model = tf.keras.models.load_model("models/segmentation_model")

# # Cell 6: Run prediction
# prediction = model.predict(input_tensor)[0]  # shape: (H, W, C) or (H, W)
# mask = np.argmax(prediction, axis=-1) if prediction.ndim == 3 else prediction
# mask = mask.astype(np.uint8)

# # Resize mask back to original size (optional)
# mask_resized = cv2.resize(mask, (original_img.shape[1], original_img.shape[0]), interpolation=cv2.INTER_NEAREST)

# # Cell 7: Visualize segmentation mask
# plt.figure(figsize=(6, 6))
# plt.imshow(mask_resized, cmap='jet')
# plt.title("Predicted Segmentation Mask")
# plt.axis("off")
# plt.show()

# # Cell 8: Overlay mask on original image
# def overlay_mask(image, mask, alpha=0.5, colormap=cv2.COLORMAP_JET):
#     color_mask = cv2.applyColorMap(mask * 10, colormap)  # scale for color map
#     overlay = cv2.addWeighted(image, 1 - alpha, color_mask, alpha, 0)
#     return overlay

# overlay = overlay_mask(original_img, mask_resized)

# plt.figure(figsize=(6, 6))
# plt.imshow(overlay)
# plt.title("Overlay: Image + Mask")
# plt.axis("off")
# plt.show()


In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Install TF Hub if needed
# !pip install tensorflow tensorflow-hub

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
# Load DeepLabV3+ pretrained model (MobileNetV2 backbone)
model = hub.load("https://tfhub.dev/tensorflow/deeplabv3-mobilenetv2-cityscapes/1")

TypeError: load() got an unexpected keyword argument 'verify'

In [ ]:
import urllib.request
image_url = "https://raw.githubusercontent.com/mcordts/cityscapesScripts/master/cityscapesscripts/helpers/example.png"
urllib.request.urlretrieve(image_url, "cityscapes_sample.png")


In [ ]:
def preprocess_cityscapes_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    original_size = tf.shape(image)[:2]
    resized_image = tf.image.resize(image, [513, 513])
    return image, resized_image, original_size

orig_img, input_img, orig_size = preprocess_cityscapes_image("cityscapes_sample.png")

# Run inference
result = model(input_img[tf.newaxis, ...])
seg_map = tf.argmax(result['default'][0], axis=-1)
seg_map = tf.image.resize(seg_map[..., tf.newaxis], orig_size, method='nearest').numpy().squeeze()


In [ ]:
# Resize mask back to original size (optional)
mask_resized = cv2.resize(seg_map, (original_img.shape[1], original_img.shape[0]), interpolation=cv2.INTER_NEAREST)

# Cell 7: Visualize segmentation mask
plt.figure(figsize=(6, 6))
plt.imshow(mask_resized, cmap='jet')
plt.title("Predicted Segmentation Mask")
plt.axis("off")
plt.show()

# Cell 8: Overlay mask on original image
def overlay_mask(image, mask, alpha=0.5, colormap=cv2.COLORMAP_JET):
    color_mask = cv2.applyColorMap(mask * 10, colormap)  # scale for color map
    overlay = cv2.addWeighted(image, 1 - alpha, color_mask, alpha, 0)
    return overlay

overlay = overlay_mask(original_img, mask_resized)

plt.figure(figsize=(6, 6))
plt.imshow(overlay)
plt.title("Overlay: Image + Mask")
plt.axis("off")
plt.show()